<a href="https://colab.research.google.com/github/marliyehez/Data-Engineering/blob/main/pyspark_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pyspark

In [ ]:
!pip install pyspark

# Titanic

  0% 0.00/34.1k [00:00<?, ?B/s]
100% 34.1k/34.1k [00:00<00:00, 33.8MB/s]


In [ ]:
# change kaggle config dir
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

# download the data via api
!kaggle competitions download -c titanic

# unzip the data
!unzip titanic.zip

In [105]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# Spark session
spark = SparkSession.builder \
    .appName('Kaggle Titanic') \
    .getOrCreate()


# Spark read
csv_path = '/content/train.csv'
df = spark.read.csv(csv_path, header=True, inferSchema=True)

In [106]:
df.show(3)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
only showing top 3 rows



In [117]:
# df.approxQuantile('Age', [0.5], 0.01)[0]
df.approxQuantile('SibSp', [0.5], 0.01)[0]

0.0

In [70]:
df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Height: string (nullable = true)
 |-- Weight: string (nullable = true)
 |-- Team: string (nullable = true)
 |-- NOC: string (nullable = true)
 |-- Games: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Season: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Sport: string (nullable = true)
 |-- Event: string (nullable = true)
 |-- Medal: string (nullable = true)



In [100]:
# Change NA into DNW (Do Not Win)
df = df.withColumn("Medal", F.when(F.col("Medal") == "NA", "DNW").otherwise(F.col("Medal"))).show(10)

+---+--------------------+---+---+------+------+--------------+---+-----------+----+------+-----------+-------------+--------------------+-----+
| ID|                Name|Sex|Age|Height|Weight|          Team|NOC|      Games|Year|Season|       City|        Sport|               Event|Medal|
+---+--------------------+---+---+------+------+--------------+---+-----------+----+------+-----------+-------------+--------------------+-----+
|  1|           A Dijiang|  M| 24| 180.0|  80.0|         China|CHN|1992 Summer|1992|Summer|  Barcelona|   Basketball|Basketball Men's ...|  DNW|
|  2|            A Lamusi|  M| 23| 170.0|  60.0|         China|CHN|2012 Summer|2012|Summer|     London|         Judo|Judo Men's Extra-...|  DNW|
|  3| Gunnar Nielsen Aaby|  M| 24|  null|  null|       Denmark|DEN|1920 Summer|1920|Summer|  Antwerpen|     Football|Football Men's Fo...|  DNW|
|  4|Edgar Lindenau Aabye|  M| 34|  null|  null|Denmark/Sweden|DEN|1900 Summer|1900|Summer|      Paris|   Tug-Of-War|Tug-Of-War Me

In [85]:
df.select(
    [F.sum(F.col(c).isNull().cast('int')).alias(c) for c in df.columns]
).show()

+-------+---------+--------+--------+-----------+-----------+---------+--------+----------+---------+-----------+---------+----------+----------+----------+
|IDCount|NameCount|SexCount|AgeCount|HeightCount|WeightCount|TeamCount|NOCCount|GamesCount|YearCount|SeasonCount|CityCount|SportCount|EventCount|MedalCount|
+-------+---------+--------+--------+-----------+-----------+---------+--------+----------+---------+-----------+---------+----------+----------+----------+
|      0|        0|       0|       0|          0|          0|        0|       0|         0|        0|          0|        0|         0|         0|         0|
+-------+---------+--------+--------+-----------+-----------+---------+--------+----------+---------+-----------+---------+----------+----------+----------+



+---+----+---+-----+------+------+----+---+-----+----+------+----+-----+-----+-----+
| ID|Name|Sex|  Age|Height|Weight|Team|NOC|Games|Year|Season|City|Sport|Event|Medal|
+---+----+---+-----+------+------+----+---+-----+----+------+----+-----+-----+-----+
|  0|   0|  0|10145| 60006| 62857|   0|  0|    0| 673|     0|   0|    0|    0|    0|
+---+----+---+-----+------+------+----+---+-----+----+------+----+-----+-----+-----+



+---+--------------------+---+---+------+------+--------------+---+-----------+----+------+-----------+-------------+--------------------+-----+
| ID|                Name|Sex|Age|Height|Weight|          Team|NOC|      Games|Year|Season|       City|        Sport|               Event|Medal|
+---+--------------------+---+---+------+------+--------------+---+-----------+----+------+-----------+-------------+--------------------+-----+
|  1|           A Dijiang|  M| 24| 180.0|  80.0|         China|CHN|1992 Summer|1992|Summer|  Barcelona|   Basketball|Basketball Men's ...|  DNW|
|  2|            A Lamusi|  M| 23| 170.0|  60.0|         China|CHN|2012 Summer|2012|Summer|     London|         Judo|Judo Men's Extra-...|  DNW|
|  3| Gunnar Nielsen Aaby|  M| 24|  null|  null|       Denmark|DEN|1920 Summer|1920|Summer|  Antwerpen|     Football|Football Men's Fo...|  DNW|
|  4|Edgar Lindenau Aabye|  M| 34|  null|  null|Denmark/Sweden|DEN|1900 Summer|1900|Summer|      Paris|   Tug-Of-War|Tug-Of-War Me

In [18]:
df.show(5)

+---+--------------------+---+---+------+------+--------------+---+-----------+----+------+---------+-------------+--------------------+-----+
| ID|                Name|Sex|Age|Height|Weight|          Team|NOC|      Games|Year|Season|     City|        Sport|               Event|Medal|
+---+--------------------+---+---+------+------+--------------+---+-----------+----+------+---------+-------------+--------------------+-----+
|  1|           A Dijiang|  M| 24|   180|    80|         China|CHN|1992 Summer|1992|Summer|Barcelona|   Basketball|Basketball Men's ...|   NA|
|  2|            A Lamusi|  M| 23|   170|    60|         China|CHN|2012 Summer|2012|Summer|   London|         Judo|Judo Men's Extra-...|   NA|
|  3| Gunnar Nielsen Aaby|  M| 24|    NA|    NA|       Denmark|DEN|1920 Summer|1920|Summer|Antwerpen|     Football|Football Men's Fo...|   NA|
|  4|Edgar Lindenau Aabye|  M| 34|    NA|    NA|Denmark/Sweden|DEN|1900 Summer|1900|Summer|    Paris|   Tug-Of-War|Tug-Of-War Men's ...| Gold|

# Transjakarta

In [ ]:
!pip install google-cloud-bigquery

In [ ]:
# Create a SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, monotonically_increasing_id, concat, concat_ws, create_map, lit
from itertools import chain

spark = SparkSession.builder \
    .appName('Transjakarta ETL') \
    .getOrCreate()

# Set the Google Cloud project ID
project_id = "my-project-390806"
spark.conf.set("spark.datasource.bigquery.projectId", project_id)

# Set the Google Cloud service account key file path
spark.conf.set("spark.datasource.bigquery.keyfile", "/content/my-project-390806-8bdde486c5da.json")
spark.conf.set("spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation", "true")

# Get csv files
import os
directory = "./data"
csv_files = [filename for filename in os.listdir(directory) if filename.endswith('.csv')]


# Initialize an empty list to store DataFrames
dataframes = []

# Loop through the CSV files and read them into DataFrames
for filename in csv_files:
    file_path = os.path.join(directory, filename)
    df = spark.read.csv(file_path, header=True, inferSchema=True)
    dataframes.append(df)

# Merge the DataFrames using union()
fact_table = dataframes[0]
for df in dataframes[1:]:
    fact_table = fact_table.union(df)

# Sort based on date
fact_table = fact_table.orderBy(col("tahun").asc(), col("bulan").asc())

# Dimension tables
date_dim = fact_table.select("bulan", "tahun") \
    .dropDuplicates() \
    .orderBy(col("tahun").asc(), col("bulan").asc()) \
    .withColumn("id", monotonically_increasing_id()) \
    .select("id", "bulan", "tahun")

route_dim = fact_table.select("kode_trayek", "trayek", "jenis") \
    .dropDuplicates() \
    .withColumn("id", monotonically_increasing_id()) \
    .select("id", "kode_trayek", "trayek", "jenis")

vehicle_dim = route_dim.select("jenis") \
    .dropDuplicates() \
    .withColumn("id", monotonically_increasing_id()) \
    .select("id", "jenis")


# Foreign keys dict
vehicle_dict = vehicle_dim.select("jenis", "id").rdd.collectAsMap()
route_dict = route_dim.select("kode_trayek", "id").rdd.collectAsMap()
date_dict = date_dim.withColumn(
    "bulan_tahun",
    concat_ws(
        "_",
        col("bulan"),
        col("tahun")
    )
).select("bulan_tahun", "id").rdd.collectAsMap()

# Creating foregin keys
mapping_expr = create_map([lit(x) for x in chain(*vehicle_dict.items())])
route_dim = route_dim.withColumn("vehicle_fk", mapping_expr.__getitem__(col("jenis")))

mapping_expr = create_map([lit(x) for x in chain(*route_dict.items())])
fact_table = fact_table.withColumn("route_fk", mapping_expr.__getitem__(col("kode_trayek")))

mapping_expr = create_map([lit(x) for x in chain(*date_dict.items())])
fact_table = fact_table.withColumn(
    "bulan_tahun",
    concat_ws(
        "_",
        col("bulan"),
        col("tahun")
    )
).withColumn("date_fk", mapping_expr.__getitem__(col("bulan_tahun")))


# Drop columns
cols = ["tahun", "bulan", "jenis", "kode_trayek", "trayek", "bulan_tahun"]
fact_table = fact_table.drop(*cols)
route_dim = route_dim.drop("jenis")


# Define the BigQuery dataset and table name
dataset_name = "transjakarta"
tables_list = [
    (fact_table,'fact table'),
    (route_dim, 'route dim'),
    (date_dim, 'date dim'),
    (vehicle_dim, 'vehicle dim'),
]

AnalysisException: ignored

In [ ]:
# Write the DataFrame to BigQuery
try:

    for df, table_name in tables_list:
        table_name = 'fact table'
        fact_table.write \
            .format("bigquery") \
            .option("table", f"{project_id}.{dataset_name}.{table_name}") \
            .mode("overwrite") \
            .save()
except Exception as e:
    print(e)

In [ ]:
### GCS CODE

from pyspark.sql.functions import col, monotonically_increasing_id, concat, concat_ws, create_map, lit
from itertools import chain
import gcsfs


# Set the GCS bucket and folder path
BUCKET = "transjakarta-data"
FOLDER_PATH = "raw-data"

# Create a file system object
fs = gcsfs.GCSFileSystem()

# List all the csv files in the folder
files = fs.ls(f"{BUCKET}/{FOLDER_PATH}")
csv_files = [f"gs://{file}" for file in files if file.endswith('.csv')]



# Initialize an empty list to store DataFrames
dataframes = []

# Loop through the CSV files and read them into DataFrames
for filename in csv_files:
    df = spark.read.csv(filename, header=True, inferSchema=True)
    dataframes.append(df)

# Merge the DataFrames using union()
fact_table = dataframes[0]
for df in dataframes[1:]:
    fact_table = fact_table.union(df)

# Sort based on date
fact_table = fact_table.orderBy(col("tahun").asc(), col("bulan").asc())

# Dimension tables
date_dim = fact_table.select("bulan", "tahun") \
    .dropDuplicates() \
    .orderBy(col("tahun").asc(), col("bulan").asc()) \
    .withColumn("id", monotonically_increasing_id()) \
    .select("id", "bulan", "tahun")

route_dim = fact_table.select("kode_trayek", "trayek", "jenis") \
    .dropDuplicates() \
    .withColumn("id", monotonically_increasing_id()) \
    .select("id", "kode_trayek", "trayek", "jenis")

vehicle_dim = route_dim.select("jenis") \
    .dropDuplicates() \
    .withColumn("id", monotonically_increasing_id()) \
    .select("id", "jenis")


# Foreign keys dict
vehicle_dict = vehicle_dim.select("jenis", "id").rdd.collectAsMap()
route_dict = route_dim.select("kode_trayek", "id").rdd.collectAsMap()
date_dict = date_dim.withColumn(
    "bulan_tahun",
    concat_ws(
        "_",
        col("bulan"),
        col("tahun")
    )
).select("bulan_tahun", "id").rdd.collectAsMap()

# Creating foregin keys
mapping_expr = create_map([lit(x) for x in chain(*vehicle_dict.items())])
route_dim = route_dim.withColumn("vehicle_fk", mapping_expr.__getitem__(col("jenis")))

mapping_expr = create_map([lit(x) for x in chain(*route_dict.items())])
fact_table = fact_table.withColumn("route_fk", mapping_expr.__getitem__(col("kode_trayek")))

mapping_expr = create_map([lit(x) for x in chain(*date_dict.items())])
fact_table = fact_table.withColumn(
    "bulan_tahun",
    concat_ws(
        "_",
        col("bulan"),
        col("tahun")
    )
).withColumn("date_fk", mapping_expr.__getitem__(col("bulan_tahun")))


# Drop columns
cols = ["tahun", "bulan", "jenis", "kode_trayek", "trayek", "bulan_tahun"]
fact_table = fact_table.drop(*cols)
route_dim = route_dim.drop("jenis")


# Define the BigQuery dataset and table name
PROJECT_ID = "golden-union-392713"
DATASET_NAME = "transjakarta"
tables_list = [
    (fact_table,'fact table'),
    (route_dim, 'route dim'),
    (date_dim, 'date dim'),
    (vehicle_dim, 'vehicle dim'),
]



# Write the DataFrame to BigQuery
try:

    for df, TABLE_NAME in tables_list:
        df.write \
            .format("bigquery") \
            .option("table", f"{PROJECT_ID}.{DATASET_NAME}.{TABLE_NAME}") \
            .option("temporaryGcsBucket", "transjakarta-data") \
            .mode("overwrite") \
            .save()
except Exception as e:
    print(e)

# Spark RDD

## French Bakery

In [ ]:
!unzip '/content/french_bakery.zip'

Archive:  /content/french_bakery.zip
  inflating: Bakery sales.csv        


In [ ]:
from pyspark import SparkContext
import re

sc = SparkContext.getOrCreate()

In [ ]:
df.head()

,Unnamed: 0,date,time,ticket_number,article,Quantity,unit_price
0,0,2021-01-02,08:38,150040.0,BAGUETTE,1.0,"0,90 €"
1,1,2021-01-02,08:38,150040.0,PAIN AU CHOCOLAT,3.0,"1,20 €"
2,4,2021-01-02,09:14,150041.0,PAIN AU CHOCOLAT,2.0,"1,20 €"
3,5,2021-01-02,09:14,150041.0,PAIN,1.0,"1,15 €"
4,8,2021-01-02,09:25,150042.0,TRADITIONAL BAGUETTE,5.0,"1,20 €"


- `ticket number`: identifier for every single transaction
- `article`: name of the product sold (in French)
- `quantity`: quantity sold
- `unit_price`: price per product
- `Objective`: Forecast the sales in order to ease the production planning

In [ ]:
# Get the rdd
lines = sc.textFile("Bakery sales.csv")
rdd = lines.map(lambda x: x.split(','))

# Remove header
header = rdd.first()
rdd = rdd.filter(lambda x: x!= header)

rdd.take(1)

[['0', '2021-01-02', '08:38', '150040.0', 'BAGUETTE', '1.0', '"0', '90 €"']]

In [ ]:
# Some rows contain more element
rdd.map(lambda x: (len(x), 1)).reduceByKey(lambda x, y: x+y).collect()

[(8, 233984), (9, 21)]

In [ ]:
# PLATPREPARE6,50 & PLATPREPARE7,00
rdd.filter(lambda x: (len(x)==9)).collect()

[['137071',
  '2021-07-12',
  '08:25',
  '187148.0',
  '"PLATPREPARE6',
  '50"',
  '1.0',
  '"0',
  '00 €"'],
 ['137604',
  '2021-07-12',
  '11:30',
  '187296.0',
  '"PLATPREPARE6',
  '50"',
  '2.0',
  '"6',
  '50 €"'],
 ['137605',
  '2021-07-12',
  '11:30',
  '187296.0',
  '"PLATPREPARE5',
  '50"',
  '1.0',
  '"5',
  '50 €"'],
 ['137698',
  '2021-07-12',
  '11:52',
  '187322.0',
  '"PLATPREPARE7',
  '00"',
  '3.0',
  '"7',
  '00 €"'],
 ['138743',
  '2021-07-13',
  '10:35',
  '187606.0',
  '"PLATPREPARE7',
  '00"',
  '4.0',
  '"7',
  '00 €"'],
 ['138901',
  '2021-07-13',
  '11:17',
  '187650.0',
  '"PLATPREPARE5',
  '50"',
  '1.0',
  '"5',
  '50 €"'],
 ['138903',
  '2021-07-13',
  '11:17',
  '187650.0',
  '"PLATPREPARE6',
  '00"',
  '1.0',
  '"6',
  '00 €"'],
 ['139134',
  '2021-07-13',
  '12:19',
  '187715.0',
  '"PLATPREPARE7',
  '00"',
  '1.0',
  '"7',
  '00 €"'],
 ['139559',
  '2021-07-13',
  '19:23',
  '187833.0',
  '"PLATPREPARE7',
  '00"',
  '1.0',
  '"7',
  '00 €"'],
 ['139560'

In [ ]:
# columns transfrom
def columns_transfrom(row):
    id = row[3][:-2]
    date = row[1]
    time = row[2]
    product = row[4]

    quantity =  row[-3]
    unit_price = f"{row[-2]}.{row[-1]}"

    if len(row) == 9:
        product = f"{row[4]},{row[5]}"

    return (id, date, time, product, quantity, unit_price)

rdd = rdd.map(columns_transfrom)
rdd.take(3)

[('150040', '2021-01-02', '08:38', 'BAGUETTE', '1.0', '"0.90 €"'),
 ('150040', '2021-01-02', '08:38', 'PAIN AU CHOCOLAT', '3.0', '"1.20 €"'),
 ('150041', '2021-01-02', '09:14', 'PAIN AU CHOCOLAT', '2.0', '"1.20 €"')]

In [ ]:
# quantity column
def columns_transfrom_2(row):

    id, date, time, product, quantity, unit_price = row

    year, month, date = date.split('-')
    year, month, date = int(year), int(month), int(date)

    hour, minute = time.split(':')
    hour, minute = int(hour), int(minute)

    quantity = re.search('\d(\.|")', quantity).group()[:-1]
    quantity = int(quantity)

    unit_price = re.search(r"(\d| )\.\d+", unit_price).group()
    unit_price = float(unit_price)

    return (id, year, month, date, hour, minute, product, quantity, unit_price)

rdd = rdd.map(columns_transfrom_2)
rdd.take(3)

[('150040', 2021, 1, 2, 8, 38, 'BAGUETTE', 1, 0.9),
 ('150040', 2021, 1, 2, 8, 38, 'PAIN AU CHOCOLAT', 3, 1.2),
 ('150041', 2021, 1, 2, 9, 14, 'PAIN AU CHOCOLAT', 2, 1.2)]

In [ ]:
# Produk yang paling banyak terjual
sorted(
    rdd.map(lambda x: (x[-3], x[-2]))\
    .reduceByKey(lambda x, y: x+y).collect(),

    key=lambda x: x[1], reverse=True
)

[('TRADITIONAL BAGUETTE', 116873),
 ('CROISSANT', 28500),
 ('PAIN AU CHOCOLAT', 24548),
 ('COUPE', 23487),
 ('BANETTE', 22898),
 ('BAGUETTE', 21739),
 ('CEREAL BAGUETTE', 7509),
 ('SPECIAL BREAD', 5520),
 ('FORMULE SANDWICH', 5379),
 ('TARTELETTE', 5032),
 ('BOULE 400G', 4778),
 ('CAMPAGNE', 4326),
 ('COOKIE', 3763),
 ('ECLAIR', 3654),
 ('VIK BREAD', 3633),
 ('COMPLET', 3563),
 ('FICELLE', 3449),
 ('MOISSON', 3394),
 ('BANETTINE', 3120),
 ('BOULE 200G', 3098),
 ('PAIN BANETTE', 3043),
 ('SANDWICH COMPLET', 2914),
 ('PAIN AUX RAISINS', 2753),
 ('PAIN', 2409),
 ('CROISSANT AMANDES', 2369),
 ('KOUIGN AMANN', 2367),
 ('QUIM BREAD', 2153),
 ('CAFE OU EAU', 1966),
 ('CHAUSSON AUX POMMES', 1946),
 ('PAIN CHOCO AMANDES', 1919),
 ('BOISSON 33CL', 1904),
 ('BAGUETTE GRAINE', 1880),
 ('SAND JB EMMENTAL', 1832),
 ('BRIOCHE', 1720),
 ('GRAND FAR BRETON', 1506),
 ('TRAITEUR', 1476),
 ('SEIGLE', 1418),
 ('PARIS BREST', 1294),
 ('FINANCIER X5', 1275),
 ('DEMI BAGUETTE', 1147),
 ('FLAN', 1097),
 ('MILL

In [ ]:
# transaksi dengan pendapatan paling besar
result = rdd.map(lambda x: (x[0], x[-1]*x[-2]))\
            .reduceByKey(lambda x, y: x+y)\
            .collect()


result = sorted(result, key=lambda x: x[1], reverse=True)[:10]
print('Transaction Id \t Income')
for item in result:
    print(f"{item[0]} \t\t {item[1]} €")

Transaction Id 	 Income
194199 		 79.7 €
256455 		 64.2 €
220468 		 61.2 €
197217 		 58.6 €
198481 		 57.2 €
198573 		 54.2 €
273733 		 52.800000000000004 €
272070 		 49.9 €
229276 		 49.0 €
270964 		 48.9 €


In [ ]:
# Pendapatan per tahun
result = rdd.map(lambda x: (x[1], x[-1]*x[-2]))\
            .reduceByKey(lambda x, y: x+y)\
            .collect()

print('Year \t Income')
for item in result:
    print(f"{item[0]} \t {item[1]:.2f} €")

Year 	 Income
2022 	 246857.83 €
2021 	 294123.72 €


In [ ]:
# Pendapatan per bulan
result = rdd.map(lambda x: (x[2], x[-1]*x[-2]))\
            .reduceByKey(lambda x, y: x+y)\
            .mapValues(lambda x: x/2)\
            .collect()

print('Month \t Avg Income')
for item in sorted(result, key=lambda x: x[1], reverse=True):
    print(f"{item[0]} \t {item[1]:.2f} €")

Month 	 Avg Income
8 	 49552.65 €
7 	 42699.34 €
5 	 28485.38 €
4 	 24284.75 €
6 	 24260.28 €
9 	 21611.80 €
3 	 19706.01 €
2 	 16978.22 €
1 	 14352.58 €
10 	 10938.83 €
12 	 8875.32 €
11 	 8745.62 €


In [ ]:
# Pada jam transaksi cenderung terjadi
# Pagi      3.00 - 9.59
# Siang     10.00 - 17.59
# Malam     18.00 - 2.59

def when_to_buy(row):
    when = 'night'
    if (row[4] >= 3) and (row[4] <= 9):
        when = 'morning'
    elif (row[4] >= 10) and (row[4] <= 17):
        when = 'noon'

    return (row[0], when)

result = rdd.map(when_to_buy)\
            .reduceByKey(lambda x, y: x)\
            .map(lambda x: (x[1], 1))\
            .reduceByKey(lambda x, y: x+y)\
            .collect()

for item in result:
    print(f"{item[0]}    \t{item[1]} transcations")

noon    	93182 transcations
night    	7090 transcations
morning    	36179 transcations


In [ ]:
rdd.take(2)

[('150040', 2021, 1, 2, 8, 38, 'BAGUETTE', 1, 0.9),
 ('150040', 2021, 1, 2, 8, 38, 'PAIN AU CHOCOLAT', 3, 1.2)]

In [ ]:
# INDO INGGRIS WOYYYYYYYYYYYYYYYYYYYYY

## Instagram App Store Reviews

In [ ]:
from pyspark import SparkContext
import re

sc = SparkContext.getOrCreate()

In [ ]:
# Get the rdd
# source: https://www.kaggle.com/datasets/odins0n/top-20-play-store-app-reviews-daily-update?select=all_combined.csv
rdd = sc.textFile("Instagram.csv")

rdd.take(3)

['reviewId,content,score',
 '83c7935d-d791-4d9c-b738-c7984992491a,It is a best app,5',
 '2d56d1a3-a4d0-4482-ae45-8394d2b8951f,"I love Insta, my No1 social media app❤️",5']

In [ ]:
# Remove header
header = rdd.first()
rdd = rdd.filter(lambda x: x!= header)

# Get the information about rating and review
def parser(line):
    row = line.split(',')
    rating, review = row[-1], row[1]
    for i in range(2, len(row)-1):
        review += row[i]

    return (rating, review)

rdd = rdd.map(parser)

rdd.take(2)

[('5', 'It is a best app'), ('5', '"I love Insta my No1 social media app❤️"')]

In [ ]:
# Drop rows with missing ratings
missing_ratings = rdd.filter(lambda x: re.search('^\D', x[0])).collect()
rdd = rdd.filter(lambda x: x not in missing_ratings)

missing_ratings

[('"Cool app... Watch only what u follow... Free from politics and ads... ',
  '"Cool app... Watch only what u follow... Free from politics and ads... ')]

In [ ]:
# Dist of ratings
result = rdd.mapValues(lambda x: 1)\
            .reduceByKey(lambda x, y: x+y)\
            .collect()

for item in result:
    print(f"{int(item[0])}   {item[1]}")

1   1604
4   836
5   6643
3   526
2   391


In [ ]:
# Good ratings RDD
goodRDD = rdd.filter(lambda x: x[0] in ['4', '5'])
goodRDD.take(5)

[('5', 'It is a best app'),
 ('5', '"I love Insta my No1 social media app❤️"'),
 ('5', '"THIS IS A VERY GOOD APP 👌🙂"'),
 ('5', 'Nice app'),
 ('5', 'Good reel and good dance')]

In [ ]:
rdd.filter(lambda x: '5' in x[1]).take(3)

[('5', '"No much talk I suppose give 7 🌟 but it\'s end in 5"'),
 ('5', 'Janychowbhury05'),
 ('2',
  '"I\'ve noticed an issue with checking my notifications. I receive notifications just fine but when I enter the app I cannot see what I got especially messages. You\'d think it\'s an internet issue but it happens with or without wifi even with data even with 5G. It\'s bad enough getting spam accounts and bots harassing you in the comments. There are other little bugs here and there too such as when you save a post and it says ""saved"" but it stays on for a while or even appears stuck."')]

In [ ]:
# Flatmap to normalize words
goodRDD = goodRDD.flatMap(lambda x: re.compile(r'\W+').split(x[1].lower()))

# Results
sorted(
    goodRDD.map(lambda x: (x, 1))\
        .reduceByKey(lambda x, y: x + y)\
        .collect(),

    key=lambda x: x[1], reverse=True
)[:50]

[('', 2624),
 ('app', 1710),
 ('good', 1342),
 ('i', 1080),
 ('nice', 1068),
 ('very', 897),
 ('it', 768),
 ('is', 669),
 ('and', 623),
 ('instagram', 616),
 ('this', 613),
 ('to', 552),
 ('the', 535),
 ('best', 505),
 ('my', 471),
 ('love', 463),
 ('a', 388),
 ('s', 365),
 ('you', 318),
 ('for', 313),
 ('but', 270),
 ('so', 262),
 ('super', 256),
 ('like', 232),
 ('please', 215),
 ('of', 206),
 ('great', 203),
 ('amazing', 198),
 ('not', 197),
 ('me', 188),
 ('in', 184),
 ('aap', 158),
 ('t', 153),
 ('awesome', 146),
 ('can', 141),
 ('excellent', 137),
 ('have', 134),
 ('that', 134),
 ('with', 130),
 ('use', 119),
 ('on', 106),
 ('social', 103),
 ('hai', 102),
 ('insta', 99),
 ('your', 98),
 ('application', 95),
 ('are', 92),
 ('all', 92),
 ('really', 90),
 ('time', 89)]

In [ ]:
# Bad ratings RDD
badRDD = rdd.filter(lambda x: x[0] in ['1', '2'])
badRDD.take(5)

[('1',
  "I would have given you 0 stars if i could. Fix your algorithm I'm being spammed by sex workers acounts with sketchy Links that Instagram don't see a problem in when I report it. I'm starting to hate this app and it's stupid algorithm. Just make the mobile version like the desktop version and stop this shitfest!"),
 ('1', 'Worst I want to uninstall this'),
 ('1', 'Reply problem please solved this problem'),
 ('1', "It's hanging"),
 ('1',
  '"Internet connectivity gets bugged way too often  re-downloading fixes it but not for long and you\'ll have to do the same again over and over. Ok so now re-downloading also doesn\'t help very nice 🙂"')]

In [ ]:
# Flatmap to normalize words
badRDD = badRDD.flatMap(lambda x: re.compile(r'\W+').split(x[1].lower()))

# Results
sorted(
    badRDD.map(lambda x: (x, 1))\
        .reduceByKey(lambda x, y: x + y)\
        .collect(),

    key=lambda x: x[1], reverse=True
)[:50]

[('', 1045),
 ('i', 963),
 ('to', 634),
 ('the', 590),
 ('it', 564),
 ('my', 553),
 ('and', 544),
 ('not', 483),
 ('app', 460),
 ('is', 455),
 ('t', 423),
 ('instagram', 354),
 ('this', 327),
 ('a', 315),
 ('in', 264),
 ('of', 245),
 ('account', 220),
 ('can', 213),
 ('but', 210),
 ('for', 207),
 ('s', 195),
 ('on', 194),
 ('you', 191),
 ('please', 182),
 ('me', 179),
 ('are', 152),
 ('very', 150),
 ('that', 148),
 ('have', 143),
 ('good', 140),
 ('so', 137),
 ('problem', 130),
 ('don', 129),
 ('no', 124),
 ('many', 120),
 ('as', 117),
 ('with', 113),
 ('all', 109),
 ('update', 107),
 ('working', 106),
 ('when', 104),
 ('even', 101),
 ('bad', 101),
 ('option', 99),
 ('there', 95),
 ('am', 93),
 ('like', 89),
 ('post', 85),
 ('nice', 84),
 ('now', 83)]

## Romance Book

In [ ]:
from pyspark import SparkContext
import re

sc = SparkContext.getOrCreate()

In [ ]:
rdd = sc.textFile("romance.txt")

# Flatmap to normalize words
rdd = rdd.flatMap(lambda x: re.compile(r'\W+').split(x.lower()))

In [ ]:
sorted(
    rdd.map(lambda x: (x, 1))\
        .reduceByKey(lambda x, y: x + y)\
        .collect(),

    key=lambda x: x[1], reverse=True
)[:30]

[('', 5086),
 ('the', 877),
 ('and', 806),
 ('i', 659),
 ('to', 626),
 ('a', 547),
 ('of', 518),
 ('in', 394),
 ('is', 372),
 ('that', 369),
 ('you', 368),
 ('my', 356),
 ('s', 324),
 ('romeo', 320),
 ('with', 300),
 ('not', 288),
 ('thou', 278),
 ('this', 266),
 ('me', 266),
 ('for', 251),
 ('it', 244),
 ('d', 242),
 ('be', 227),
 ('juliet', 193),
 ('but', 185),
 ('what', 174),
 ('o', 173),
 ('thy', 170),
 ('as', 168),
 ('capulet', 163)]

# Spark SQL

## Go to College?

In [ ]:
from pyspark.sql import SparkSession, functions
spark = SparkSession.builder.appName('app').getOrCreate()

In [ ]:
df = spark.read.csv('go_to_college.csv', header=True, inferSchema=True)

Columns' Explanation




> `type_school` : Type of school that student attends\
> `school_accreditation` : Quality of school. A is better than B.\
> `gender` : Gender of student\
> `interest` : How interested are students if they go to college\
> `residence` : Type of residence\
> `parent_age` : Parent age\
> `parent_salary` : Parent salary per month in IDR/Rupiah\
> `house_area` : Parent house area in meter square\
> `average_grades` : Average of grades in scale of 0-100\
> `parent_was_in_college` : Was parent ever in college?\
> `will_go_to_college` : Predictions about going to college\

In [ ]:
df.printSchema()

root
 |-- type_school: string (nullable = true)
 |-- school_accreditation: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- interest: string (nullable = true)
 |-- residence: string (nullable = true)
 |-- parent_age: integer (nullable = true)
 |-- parent_salary: integer (nullable = true)
 |-- house_area: double (nullable = true)
 |-- average_grades: double (nullable = true)
 |-- parent_was_in_college: boolean (nullable = true)
 |-- will_go_to_college: boolean (nullable = true)



In [ ]:
df.dtypes

[('type_school', 'string'),
 ('school_accreditation', 'string'),
 ('gender', 'string'),
 ('interest', 'string'),
 ('residence', 'string'),
 ('parent_age', 'int'),
 ('parent_salary', 'int'),
 ('house_area', 'double'),
 ('average_grades', 'double'),
 ('parent_was_in_college', 'boolean'),
 ('will_go_to_college', 'boolean')]

In [ ]:
cols = ['parent_age', 'parent_salary', 'house_area', 'average_grades']
# df.select(cols).describe().show()
df.select(cols).summary().show()

+-------+-----------------+------------------+------------------+------------------+
|summary|       parent_age|     parent_salary|        house_area|    average_grades|
+-------+-----------------+------------------+------------------+------------------+
|  count|             1000|              1000|              1000|              1000|
|   mean|           52.208|         5381570.0| 74.51530000000005| 86.09719999999999|
| stddev|3.500426972383368|1397545.9096822797|15.293345687989016|3.3787384085236942|
|    min|               40|           1000000|              20.0|              75.0|
|    25%|               50|           4360000|              64.6|             83.73|
|    50%|               52|           5440000|              75.5|             85.57|
|    75%|               54|           6380000|              84.8|             88.26|
|    max|               65|          10000000|             120.0|              98.0|
+-------+-----------------+------------------+------------------+

### Categorical Columns

**Result**

> Many of `Very Interested` and `Interested` student are not going to college. The opposite also happened to the  `Not Interested` and `Less Interested` student. We can see that interest doesn't linearly correlated with college admission. Even so, the presence of significance can make us use `interest` column as a predictor.

> Other categorical columns don't give siginificant difference in affecting student to go to college.

In [ ]:
# interest
col = 'interest'
for value in df.select(col).distinct().collect():

    print(value[0])
    df.filter(df[col]==value[0])\
        .groupBy('will_go_to_college')\
        .count()\
        .show()

Very Interested
+------------------+-----+
|will_go_to_college|count|
+------------------+-----+
|              true|  190|
|             false|  134|
+------------------+-----+

Less Interested
+------------------+-----+
|will_go_to_college|count|
+------------------+-----+
|              true|   87|
|             false|  142|
+------------------+-----+

Uncertain
+------------------+-----+
|will_go_to_college|count|
+------------------+-----+
|              true|  158|
|             false|  103|
+------------------+-----+

Interested
+------------------+-----+
|will_go_to_college|count|
+------------------+-----+
|              true|   19|
|             false|   81|
+------------------+-----+

Not Interested
+------------------+-----+
|will_go_to_college|count|
+------------------+-----+
|              true|   46|
|             false|   40|
+------------------+-----+



In [ ]:
# type_school
col = 'type_school'
for value in df.select(col).distinct().collect():

    print(value[0])
    df.filter(df[col]==value[0])\
        .groupBy('will_go_to_college')\
        .count()\
        .show()

Vocational
+------------------+-----+
|will_go_to_college|count|
+------------------+-----+
|              true|  187|
|             false|  204|
+------------------+-----+

Academic
+------------------+-----+
|will_go_to_college|count|
+------------------+-----+
|              true|  313|
|             false|  296|
+------------------+-----+



In [ ]:
# school_accreditation
col = 'school_accreditation'
for value in df.select(col).distinct().collect():

    print(value[0])
    df.filter(df[col]==value[0])\
        .groupBy('will_go_to_college')\
        .count()\
        .show()

B
+------------------+-----+
|will_go_to_college|count|
+------------------+-----+
|              true|  252|
|             false|  267|
+------------------+-----+

A
+------------------+-----+
|will_go_to_college|count|
+------------------+-----+
|              true|  248|
|             false|  233|
+------------------+-----+



In [ ]:
# gender
col = 'gender'
for value in df.select(col).distinct().collect():

    print(value[0])
    df.filter(df[col]==value[0])\
        .groupBy('will_go_to_college')\
        .count()\
        .show()

Female
+------------------+-----+
|will_go_to_college|count|
+------------------+-----+
|              true|  251|
|             false|  234|
+------------------+-----+

Male
+------------------+-----+
|will_go_to_college|count|
+------------------+-----+
|              true|  249|
|             false|  266|
+------------------+-----+



In [ ]:
# residence
col = 'residence'
for value in df.select(col).distinct().collect():

    print(value[0])
    df.filter(df[col]==value[0])\
        .groupBy('will_go_to_college')\
        .count()\
        .show()

Urban
+------------------+-----+
|will_go_to_college|count|
+------------------+-----+
|              true|  271|
|             false|  268|
+------------------+-----+

Rural
+------------------+-----+
|will_go_to_college|count|
+------------------+-----+
|              true|  229|
|             false|  232|
+------------------+-----+



In [ ]:
# parent_was_in_college
col = 'parent_was_in_college'
for value in df.select(col).distinct().collect():

    print(value[0])
    df.filter(df[col]==value[0])\
        .groupBy('will_go_to_college')\
        .count()\
        .show()

True
+------------------+-----+
|will_go_to_college|count|
+------------------+-----+
|              true|  272|
|             false|  248|
+------------------+-----+

False
+------------------+-----+
|will_go_to_college|count|
+------------------+-----+
|              true|  228|
|             false|  252|
+------------------+-----+



### Numerical Columns

**Result**


> 1.   Good grade students tend to go to college and bad grade students tend not to go to college.
2.   Old and young parents tend to hold their children from going to college.
3.   Parents' income and their home are positively correlated with sending their children to college.

>*Warning* :\
We have not examined the possibility of multicollinearity.



In [ ]:
# Numerical columns + summary
cols = ['summary', 'parent_age', 'parent_salary', 'house_area', 'average_grades']

# Students who will go to college
print('Students who will go to college')
df.filter(df['will_go_to_college']=='true').summary().select(cols).show()

# Students who wont go to college
print('Students who wont go to college')
df.filter(df['will_go_to_college']=='false').summary().select(cols).show()

Students who will go to college
+-------+------------------+-----------------+-----------------+-----------------+
|summary|        parent_age|    parent_salary|       house_area|   average_grades|
+-------+------------------+-----------------+-----------------+-----------------+
|  count|               500|              500|              500|              500|
|   mean|            52.358|        6046040.0|81.65779999999991|87.80306000000002|
| stddev|2.9966307065072875|1212242.129428243|12.84237877283995|3.568375125806835|
|    min|                41|          1000000|             40.9|             75.0|
|    25%|                51|          5300000|             74.4|            85.41|
|    50%|                52|          6060000|             81.3|            87.64|
|    75%|                54|          6730000|             90.1|            90.28|
|    max|                64|         10000000|            120.0|             98.0|
+-------+------------------+-----------------+---------

In [ ]:
# Good grades
filt = df['average_grades'] > 90
df.filter(filt).groupBy('will_go_to_college').count().show()

+------------------+-----+
|will_go_to_college|count|
+------------------+-----+
|              true|  131|
|             false|    2|
+------------------+-----+



In [ ]:
# Bad grades
filt = df['average_grades'] < 83
df.filter(filt).groupBy('will_go_to_college').count().show()

+------------------+-----+
|will_go_to_college|count|
+------------------+-----+
|              true|   39|
|             false|  117|
+------------------+-----+



In [ ]:
# Old parents
filt = df['parent_age'] > 55
df.filter(filt).groupBy('will_go_to_college').count().show()

+------------------+-----+
|will_go_to_college|count|
+------------------+-----+
|              true|   59|
|             false|   92|
+------------------+-----+



In [ ]:
# Young parents
filt = df['parent_age'] < 48
df.filter(filt).groupBy('will_go_to_college').count().show()

+------------------+-----+
|will_go_to_college|count|
+------------------+-----+
|              true|   25|
|             false|   64|
+------------------+-----+



In [ ]:
# Middle age parents
filt = (df['parent_age'] > 50) & (df['parent_age'] < 54)
df.filter(filt).groupBy('will_go_to_college').count().show()

+------------------+-----+
|will_go_to_college|count|
+------------------+-----+
|              true|  210|
|             false|  154|
+------------------+-----+



In [ ]:
# Quartile of parent_salary
q1, q2, q3 = df.approxQuantile("parent_salary", [0.25, 0.5, 0.75], 0)
q1, q2, q3

(4360000.0, 5440000.0, 6380000.0)

In [ ]:
# Relative low income salary
filt = df['parent_salary'] < q1
df.filter(filt).groupBy('will_go_to_college').count().show()

+------------------+-----+
|will_go_to_college|count|
+------------------+-----+
|              true|   36|
|             false|  213|
+------------------+-----+



In [ ]:
# Relative high income salary
filt = df['parent_salary'] > q3
df.filter(filt).groupBy('will_go_to_college').count().show()

+------------------+-----+
|will_go_to_college|count|
+------------------+-----+
|              true|  197|
|             false|   53|
+------------------+-----+



In [ ]:
# Relative middle income salary
filt = (df['parent_salary'] > q1) & (df['parent_salary'] < q3)
df.filter(filt).groupBy('will_go_to_college').count().show()

+------------------+-----+
|will_go_to_college|count|
+------------------+-----+
|              true|  266|
|             false|  232|
+------------------+-----+



In [ ]:
# Median or higher salary
filt = df['parent_salary'] > q2
df.filter(filt).groupBy('will_go_to_college').count().show()

+------------------+-----+
|will_go_to_college|count|
+------------------+-----+
|              true|  353|
|             false|  146|
+------------------+-----+



In [ ]:
# Quartile of house area
q1, q2, q3 = df.approxQuantile("house_area", [0.25, 0.5, 0.75], 0)
q1, q2, q3

(64.6, 75.5, 84.8)

In [ ]:
# Small house_area
filt = df['house_area'] < q1
df.filter(filt).groupBy('will_go_to_college').count().show()

+------------------+-----+
|will_go_to_college|count|
+------------------+-----+
|              true|   46|
|             false|  200|
+------------------+-----+



In [ ]:
# Large house area
filt = df['house_area'] > q3
df.filter(filt).groupBy('will_go_to_college').count().show()

+------------------+-----+
|will_go_to_college|count|
+------------------+-----+
|              true|  198|
|             false|   52|
+------------------+-----+



In [ ]:
# Middle house area
filt = (df['house_area'] > q1) & (df['house_area'] < q3)
df.filter(filt).groupBy('will_go_to_college').count().show()

+------------------+-----+
|will_go_to_college|count|
+------------------+-----+
|              true|  253|
|             false|  243|
+------------------+-----+



In [ ]:
# Median or larger house area
filt = df['house_area'] > q2
df.filter(filt).groupBy('will_go_to_college').count().show()

+------------------+-----+
|will_go_to_college|count|
+------------------+-----+
|              true|  350|
|             false|  148|
+------------------+-----+



# KJ + Notes Udemy


## Section 2
Spark Basics and the RDD Interface

`keys` itu identifier yang berada di paling kiri.

### ratings-counter.py


data yg akan diproses:\
(user id, movie id, rating value, timestamp)

```
196 242 3 881250949
186 302 3 891717742
22  377 1 878887116
244  51 2 880606923
166 346 1 886397596
 .   .  .     .
 .   .  .     .
 .   .  .     .
```

In [ ]:
from pyspark import SparkConf, SparkContext
import collections

conf = SparkConf().setMaster("local").setAppName("RatingsHistogram")
sc = SparkContext(conf = conf)

lines = sc.textFile("file:///SparkCourse/ml-100k/u.data")
ratings = lines.map(lambda x: x.split()[2])
result = ratings.countByValue()

sortedResults = collections.OrderedDict(sorted(result.items()))
for key, value in sortedResults.items():
    print("%s %i" % (key, value))

In [ ]:
# penjelasan `sorted(result.items())`
result = dict(zip([3,1,2], [2,2,1]))

print(result)
sorted(result.items())

{3: 2, 1: 2, 2: 1}


[(1, 2), (2, 1), (3, 2)]

### friends-by-age.py

(id, name, age, number_of_friends)

In [ ]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("FriendsByAge")
sc = SparkContext(conf = conf)

def parseLine(line):
    fields = line.split(',')
    age = int(fields[2])
    numFriends = int(fields[3])
    return (age, numFriends)

lines = sc.textFile("file:///SparkCourse/fakefriends.csv")
rdd = lines.map(parseLine)
totalsByAge = rdd.mapValues(lambda x: (x, 1)).reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
averagesByAge = totalsByAge.mapValues(lambda x: x[0] / x[1])
results = averagesByAge.collect()
for result in results:
    print(result)


### min-temperatures.py

(weather station id, date, jens faktor iklim, nilai faktor iklim, data tidak berguna)

```
ITE00100554,18000101,TMAX,-75,,,E,
ITE00100554,18000101,TMIN,-148,,,E,
GM000010962,18000101,PRCP,0,,,E,
     .          .     .       .
     .          .     .       .
     .          .     .       .
```

In [ ]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("MinTemperatures")
sc = SparkContext(conf = conf)

def parseLine(line):
    fields = line.split(',')
    stationID = fields[0]
    entryType = fields[2]
    temperature = float(fields[3]) * 0.1 * (9.0 / 5.0) + 32.0 # convert to F from C
    return (stationID, entryType, temperature)

lines = sc.textFile("file:///SparkCourse/1800.csv")
parsedLines = lines.map(parseLine)
minTemps = parsedLines.filter(lambda x: "TMIN" in x[1])
stationTemps = minTemps.map(lambda x: (x[0], x[2]))
minTemps = stationTemps.reduceByKey(lambda x, y: min(x,y))
results = minTemps.collect();

for result in results:
    print(result[0] + "\t{:.2f}F".format(result[1]))

### max-temperatures.py

In [ ]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("MaxTemperatures")
sc = SparkContext(conf = conf)

def parseLine(line):
    fields = line.split(',')
    stationID = fields[0]
    entryType = fields[2]
    temperature = float(fields[3]) * 0.1 * (9.0 / 5.0) + 32.0
    return (stationID, entryType, temperature)

lines = sc.textFile("file:///SparkCourse/1800.csv")
parsedLines = lines.map(parseLine)
maxTemps = parsedLines.filter(lambda x: "TMAX" in x[1])
stationTemps = maxTemps.map(lambda x: (x[0], x[2]))
maxTemps = stationTemps.reduceByKey(lambda x, y: max(x,y))
results = maxTemps.collect();

for result in results:
    print(result[0] + "\t{:.2f}F".format(result[1]))

### word-count.py

In [ ]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("WordCount")
sc = SparkContext(conf = conf)

input = sc.textFile("file:///sparkcourse/book.txt")
words = input.flatMap(lambda x: x.split())
wordCounts = words.countByValue()

for word, count in wordCounts.items():
    cleanWord = word.encode('ascii', 'ignore')
    if (cleanWord):
        print(cleanWord.decode() + " " + str(count))

### word-count-better-sorted.py

In [ ]:
import re
from pyspark import SparkConf, SparkContext

def normalizeWords(text):
    return re.compile(r'\W+', re.UNICODE).split(text.lower())

conf = SparkConf().setMaster("local").setAppName("WordCount")
sc = SparkContext(conf = conf)

input = sc.textFile("file:///sparkcourse/book.txt")
words = input.flatMap(normalizeWords)

wordCounts = words.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)
wordCountsSorted = wordCounts.map(lambda x: (x[1], x[0])).sortByKey()
results = wordCountsSorted.collect()

for result in results:
    count = str(result[0])
    word = result[1].encode('ascii', 'ignore')
    if (word):
        print(word.decode() + ":\t\t" + count)

### total-spent-by-customer-sorted.py

In [ ]:
(customer id, item id, pay amount)

In [ ]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("SpendByCustomerSorted")
sc = SparkContext(conf = conf)

def extractCustomerPricePairs(line):
    fields = line.split(',')
    return (int(fields[0]), float(fields[2]))

input = sc.textFile("file:///sparkcourse/customer-orders.csv")
mappedInput = input.map(extractCustomerPricePairs)
totalByCustomer = mappedInput.reduceByKey(lambda x, y: x + y)

#Changed for Python 3 compatibility:
#flipped = totalByCustomer.map(lambda (x,y):(y,x))
flipped = totalByCustomer.map(lambda x: (x[1], x[0]))

totalByCustomerSorted = flipped.sortByKey()

results = totalByCustomerSorted.collect();
for result in results:
    print(result)

## Section 3
SparkSQL, DataFrames, and DataSets

### spark-sql.py

(id, name, age, number_of_friends)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

def mapper(line):
    fields = line.split(',')
    return Row(ID=int(fields[0]), name=str(fields[1].encode("utf-8")), \
               age=int(fields[2]), numFriends=int(fields[3]))

lines = spark.sparkContext.textFile("fakefriends.csv")
people = lines.map(mapper)

# Infer the schema, and register the DataFrame as a table.
schemaPeople = spark.createDataFrame(people).cache()
schemaPeople.createOrReplaceTempView("people")

# SQL can be run over DataFrames that have been registered as a table.
teenagers = spark.sql("SELECT * FROM people WHERE age >= 13 AND age <= 19")

# The results of SQL queries are RDDs and support all the normal RDD operations.
for teen in teenagers.collect():
  print(teen)

# We can also use functions instead of SQL queries:
schemaPeople.groupBy("age").count().orderBy("age").show()

spark.stop()

### spark-sql-dataframe.py

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

people = spark.read.option("header", "true").option("inferSchema", "true")\
    .csv("file:///SparkCourse/fakefriends-header.csv")

print("Here is our inferred schema:")
people.printSchema()

print("Let's display the name column:")
people.select("name").show()

print("Filter out anyone over 21:")
people.filter(people.age < 21).show()

print("Group by age")
people.groupBy("age").count().show()

print("Make everyone 10 years older:")
people.select(people.name, people.age + 10).show()

spark.stop()

### friends-by-age-dataframe.py

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions as func

spark = SparkSession.builder.appName("FriendsByAge").getOrCreate()

lines = spark.read.option("header", "true").option("inferSchema", "true").csv("file:///SparkCourse/fakefriends-header.csv")

# Select only age and numFriends columns
friendsByAge = lines.select("age", "friends")

# From friendsByAge we group by "age" and then compute average
friendsByAge.groupBy("age").avg("friends").show()

# Sorted
friendsByAge.groupBy("age").avg("friends").sort("age").show()

# Formatted more nicely
friendsByAge.groupBy("age").agg(func.round(func.avg("friends"), 2)).sort("age").show()

# With a custom column name
friendsByAge.groupBy("age").agg(func.round(func.avg("friends"), 2)
  .alias("friends_avg")).sort("age").show()

spark.stop()

### word-count-better-sorted-dataframe.py

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func

spark = SparkSession.builder.appName("WordCount").getOrCreate()

# Read each line of my book into a dataframe
inputDF = spark.read.text("file:///SparkCourse/book.txt")

# Split using a regular expression that extracts words
words = inputDF.select(func.explode(func.split(inputDF.value, "\\W+")).alias("word"))
wordsWithoutEmptyString = words.filter(words.word != "")

# Normalize everything to lowercase
lowercaseWords = wordsWithoutEmptyString.select(func.lower(wordsWithoutEmptyString.word).alias("word"))

# Count up the occurrences of each word
wordCounts = lowercaseWords.groupBy("word").count()

# Sort by counts
wordCountsSorted = wordCounts.sort("count")

# Show the results.
wordCountsSorted.show(wordCountsSorted.count())


### min-temperatures-dataframe.py

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

spark = SparkSession.builder.appName("MinTemperatures").getOrCreate()

# schema dibutuhkan karena tidak dimiliki header untuk data di csv
schema = StructType([ \
                     StructField("stationID", StringType(), True), \
                     StructField("date", IntegerType(), True), \
                     StructField("measure_type", StringType(), True), \
                     StructField("temperature", FloatType(), True)]) # <- langsung dinamain, walaupun belum tentu temp

# // Read the file as dataframe
df = spark.read.schema(schema).csv("file:///SparkCourse/1800.csv")
df.printSchema()

# Filter out all but TMIN entries
minTemps = df.filter(df.measure_type == "TMIN")

# Select only stationID and temperature
stationTemps = minTemps.select("stationID", "temperature")

# Aggregate to find minimum temperature for every station
minTempsByStation = stationTemps.groupBy("stationID").min("temperature")
minTempsByStation.show()

# Convert temperature to fahrenheit and sort the dataset
minTempsByStationF = minTempsByStation.withColumn("temperature",
                                                  func.round(func.col("min(temperature)") * 0.1 * (9.0 / 5.0) + 32.0, 2))\
                                                  .select("stationID", "temperature").sort("temperature")

### witColumn berguna untuk membuat kolom baru. Nama kolom lama
###  ternyata "min(temperature)" otomatis dari pyspark pada line 24
### https://stackoverflow.com/questions/39050248/how-to-modify-transform-the-column-of-a-dataframe

# Collect, format, and print the results
results = minTempsByStationF.collect()

for result in results:
    print(result[0] + "\t{:.2f}F".format(result[1]))

spark.stop()

### total-spent-customer-sorted-dataframe.py

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType

spark = SparkSession.builder.appName("TotalSpentByCustomer").master("local[*]").getOrCreate()

# Create schema when reading customer-orders
customerOrderSchema = StructType([ \
                                  StructField("cust_id", IntegerType(), True),
                                  StructField("item_id", IntegerType(), True),
                                  StructField("amount_spent", FloatType(), True)
                                  ])

# Load up the data into spark dataset
customersDF = spark.read.schema(customerOrderSchema).csv("file:///SparkCourse/customer-orders.csv")

totalByCustomer = customersDF.groupBy("cust_id").agg(func.round(func.sum("amount_spent"), 2) \
                                      .alias("total_spent"))

totalByCustomerSorted = totalByCustomer.sort("total_spent")

totalByCustomerSorted.show(totalByCustomerSorted.count())

spark.stop()

## Section 4
AdvancedExamples of Spark Programs

### popular-movies-dataframe.py

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, IntegerType, LongType

spark = SparkSession.builder.appName("PopularMovies").getOrCreate()

# Create schema when reading u.data
schema = StructType([ \
                     StructField("userID", IntegerType(), True), \
                     StructField("movieID", IntegerType(), True), \
                     StructField("rating", IntegerType(), True), \
                     StructField("timestamp", LongType(), True)])

# Load up movie data as dataframe
moviesDF = spark.read.option("sep", "\t").schema(schema).csv("file:///SparkCourse/ml-100k/u.data")

# Some SQL-style magic to sort all movies by popularity in one line!
topMovieIDs = moviesDF.groupBy("movieID").count().orderBy(func.desc("count"))

# Grab the top 10
topMovieIDs.show(10)

# Stop the session
spark.stop()

### popular-movies-nice-dataframe.py

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Sep  7 15:28:00 2020

@author: Frank
"""

from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, IntegerType, LongType
import codecs

def loadMovieNames():
    movieNames = {}
    # CHANGE THIS TO THE PATH TO YOUR u.ITEM FILE:
    with codecs.open("E:/SparkCourse/ml-100k/u.ITEM", "r", encoding='ISO-8859-1', errors='ignore') as f:
        for line in f:
            fields = line.split('|')
            movieNames[int(fields[0])] = fields[1]
    return movieNames

spark = SparkSession.builder.appName("PopularMovies").getOrCreate()

nameDict = spark.sparkContext.broadcast(loadMovieNames())

# Create schema when reading u.data
schema = StructType([ \
                     StructField("userID", IntegerType(), True), \
                     StructField("movieID", IntegerType(), True), \
                     StructField("rating", IntegerType(), True), \
                     StructField("timestamp", LongType(), True)])

# Load up movie data as dataframe
moviesDF = spark.read.option("sep", "\t").schema(schema).csv("file:///SparkCourse/ml-100k/u.data")

movieCounts = moviesDF.groupBy("movieID").count()

# Create a user-defined function to look up movie names from our broadcasted dictionary
def lookupName(movieID):
    return nameDict.value[movieID]

lookupNameUDF = func.udf(lookupName)

# Add a movieTitle column using our new udf
moviesWithNames = movieCounts.withColumn("movieTitle", lookupNameUDF(func.col("movieID")))

# Sort the results
sortedMoviesWithNames = moviesWithNames.orderBy(func.desc("count"))

# Grab the top 10
sortedMoviesWithNames.show(10, False)

# Stop the session
spark.stop()


### most-popular-superhero-dataframe.py

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

spark = SparkSession.builder.appName("MostPopularSuperhero").getOrCreate()

schema = StructType([ \
                     StructField("id", IntegerType(), True), \
                     StructField("name", StringType(), True)])

names = spark.read.schema(schema).option("sep", " ").csv("file:///SparkCourse/Marvel-names.txt")

lines = spark.read.text("file:///SparkCourse/Marvel-graph.txt")

# Small tweak vs. what's shown in the video: we trim each line of whitespace as that could
# throw off the counts.
connections = lines.withColumn("id", func.split(func.trim(func.col("value")), " ")[0]) \
    .withColumn("connections", func.size(func.split(func.trim(func.col("value")), " ")) - 1) \
    .groupBy("id").agg(func.sum("connections").alias("connections"))

mostPopular = connections.sort(func.col("connections").desc()).first()

mostPopularName = names.filter(func.col("id") == mostPopular[0]).select("name").first()

print(mostPopularName[0] + " is the most popular superhero with " + str(mostPopular[1]) + " co-appearances.")

### most-obscure-superheroes.py

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

spark = SparkSession.builder.appName("MostObscureSuperheroes").getOrCreate()

schema = StructType([ \
                     StructField("id", IntegerType(), True), \
                     StructField("name", StringType(), True)])

names = spark.read.schema(schema).option("sep", " ").csv("file:///SparkCourse/Marvel-names.txt")

lines = spark.read.text("file:///SparkCourse/Marvel-graph.txt")

# Small tweak vs. what's shown in the video: we trim whitespace from each line as this
# could throw the counts off by one.
connections = lines.withColumn("id", func.split(func.trim(func.col("value")), " ")[0]) \
    .withColumn("connections", func.size(func.split(func.trim(func.col("value")), " ")) - 1) \
    .groupBy("id").agg(func.sum("connections").alias("connections"))

minConnectionCount = connections.agg(func.min("connections")).first()[0]

minConnections = connections.filter(func.col("connections") == minConnectionCount)

minConnectionsWithNames = minConnections.join(names, "id")

print("The following characters have only " + str(minConnectionCount) + " connection(s):")

minConnectionsWithNames.select("name").show()

### degrees-of-separation.py

In [ ]:
# Boilerplate stuff:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("DegreesOfSeparation")
sc = SparkContext(conf = conf)

# The characters we wish to find the degree of separation between:
startCharacterID = 5306 #SpiderMan
targetCharacterID = 14  #ADAM 3,031 (who?)

# Our accumulator, used to signal when we find the target character during
# our BFS traversal.
hitCounter = sc.accumulator(0)

def convertToBFS(line):
    fields = line.split()
    heroID = int(fields[0])
    connections = []
    for connection in fields[1:]:
        connections.append(int(connection))

    color = 'WHITE'
    distance = 9999

    if (heroID == startCharacterID):
        color = 'GRAY'
        distance = 0

    return (heroID, (connections, distance, color))


def createStartingRdd():
    inputFile = sc.textFile("file:///sparkcourse/marvel-graph.txt")
    return inputFile.map(convertToBFS)

def bfsMap(node):
    characterID = node[0]
    data = node[1]
    connections = data[0]
    distance = data[1]
    color = data[2]

    results = []

    #If this node needs to be expanded...
    if (color == 'GRAY'):
        for connection in connections:
            newCharacterID = connection
            newDistance = distance + 1
            newColor = 'GRAY'
            if (targetCharacterID == connection):
                hitCounter.add(1)

            newEntry = (newCharacterID, ([], newDistance, newColor))
            results.append(newEntry)

        #We've processed this node, so color it black
        color = 'BLACK'

    #Emit the input node so we don't lose it.
    results.append( (characterID, (connections, distance, color)) )
    return results

def bfsReduce(data1, data2):
    edges1 = data1[0]
    edges2 = data2[0]
    distance1 = data1[1]
    distance2 = data2[1]
    color1 = data1[2]
    color2 = data2[2]

    distance = 9999
    color = color1
    edges = []

    # See if one is the original node with its connections.
    # If so preserve them.
    if (len(edges1) > 0):
        edges.extend(edges1)
    if (len(edges2) > 0):
        edges.extend(edges2)

    # Preserve minimum distance
    if (distance1 < distance):
        distance = distance1

    if (distance2 < distance):
        distance = distance2

    # Preserve darkest color
    if (color1 == 'WHITE' and (color2 == 'GRAY' or color2 == 'BLACK')):
        color = color2

    if (color1 == 'GRAY' and color2 == 'BLACK'):
        color = color2

    if (color2 == 'WHITE' and (color1 == 'GRAY' or color1 == 'BLACK')):
        color = color1

    if (color2 == 'GRAY' and color1 == 'BLACK'):
        color = color1

    return (edges, distance, color)


#Main program here:
iterationRdd = createStartingRdd()

for iteration in range(0, 10):
    print("Running BFS iteration# " + str(iteration+1))

    # Create new vertices as needed to darken or reduce distances in the
    # reduce stage. If we encounter the node we're looking for as a GRAY
    # node, increment our accumulator to signal that we're done.
    mapped = iterationRdd.flatMap(bfsMap)

    # Note that mapped.count() action here forces the RDD to be evaluated, and
    # that's the only reason our accumulator is actually updated.
    print("Processing " + str(mapped.count()) + " values.")

    if (hitCounter.value > 0):
        print("Hit the target character! From " + str(hitCounter.value) \
            + " different direction(s).")
        break

    # Reducer combines data for each character ID, preserving the darkest
    # color and shortest path.
    iterationRdd = mapped.reduceByKey(bfsReduce)


### movie-similarities-dataframe.py

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType
import sys

def computeCosineSimilarity(spark, data):
    # Compute xx, xy and yy columns
    pairScores = data \
      .withColumn("xx", func.col("rating1") * func.col("rating1")) \
      .withColumn("yy", func.col("rating2") * func.col("rating2")) \
      .withColumn("xy", func.col("rating1") * func.col("rating2"))

    # Compute numerator, denominator and numPairs columns
    calculateSimilarity = pairScores \
      .groupBy("movie1", "movie2") \
      .agg( \
        func.sum(func.col("xy")).alias("numerator"), \
        (func.sqrt(func.sum(func.col("xx"))) * func.sqrt(func.sum(func.col("yy")))).alias("denominator"), \
        func.count(func.col("xy")).alias("numPairs")
      )

    # Calculate score and select only needed columns (movie1, movie2, score, numPairs)
    result = calculateSimilarity \
      .withColumn("score", \
        func.when(func.col("denominator") != 0, func.col("numerator") / func.col("denominator")) \
          .otherwise(0) \
      ).select("movie1", "movie2", "score", "numPairs")

    return result

# Get movie name by given movie id
def getMovieName(movieNames, movieId):
    result = movieNames.filter(func.col("movieID") == movieId) \
        .select("movieTitle").collect()[0]

    return result[0]


spark = SparkSession.builder.appName("MovieSimilarities").master("local[*]").getOrCreate()

movieNamesSchema = StructType([ \
                               StructField("movieID", IntegerType(), True), \
                               StructField("movieTitle", StringType(), True) \
                               ])

moviesSchema = StructType([ \
                     StructField("userID", IntegerType(), True), \
                     StructField("movieID", IntegerType(), True), \
                     StructField("rating", IntegerType(), True), \
                     StructField("timestamp", LongType(), True)])


# Create a broadcast dataset of movieID and movieTitle.
# Apply ISO-885901 charset
movieNames = spark.read \
      .option("sep", "|") \
      .option("charset", "ISO-8859-1") \
      .schema(movieNamesSchema) \
      .csv("file:///SparkCourse/ml-100k/u.item")

# Load up movie data as dataset
movies = spark.read \
      .option("sep", "\t") \
      .schema(moviesSchema) \
      .csv("file:///SparkCourse/ml-100k/u.data")


ratings = movies.select("userId", "movieId", "rating")

# Emit every movie rated together by the same user.
# Self-join to find every combination.
# Select movie pairs and rating pairs
moviePairs = ratings.alias("ratings1") \
      .join(ratings.alias("ratings2"), (func.col("ratings1.userId") == func.col("ratings2.userId")) \
            & (func.col("ratings1.movieId") < func.col("ratings2.movieId"))) \
      .select(func.col("ratings1.movieId").alias("movie1"), \
        func.col("ratings2.movieId").alias("movie2"), \
        func.col("ratings1.rating").alias("rating1"), \
        func.col("ratings2.rating").alias("rating2"))


moviePairSimilarities = computeCosineSimilarity(spark, moviePairs).cache()

if (len(sys.argv) > 1):
    scoreThreshold = 0.97
    coOccurrenceThreshold = 50.0

    movieID = int(sys.argv[1])

    # Filter for movies with this sim that are "good" as defined by
    # our quality thresholds above
    filteredResults = moviePairSimilarities.filter( \
        ((func.col("movie1") == movieID) | (func.col("movie2") == movieID)) & \
          (func.col("score") > scoreThreshold) & (func.col("numPairs") > coOccurrenceThreshold))

    # Sort by quality score.
    results = filteredResults.sort(func.col("score").desc()).take(10)

    print ("Top 10 similar movies for " + getMovieName(movieNames, movieID))

    for result in results:
        # Display the similarity result that isn't the movie we're looking at
        similarMovieID = result.movie1
        if (similarMovieID == movieID):
          similarMovieID = result.movie2

        print(getMovieName(movieNames, similarMovieID) + "\tscore: " \
              + str(result.score) + "\tstrength: " + str(result.numPairs))

## Section 5

### MovieSimilarities1M.py

In [ ]:
import sys
from pyspark import SparkConf, SparkContext
from math import sqrt

def loadMovieNames():
    movieNames = {}
    with open("movies.dat",  encoding='ascii', errors='ignore') as f:
        for line in f:
            fields = line.split("::")
            movieNames[int(fields[0])] = fields[1]
    return movieNames

def makePairs( userRatings ):
    ratings = userRatings[1]
    (movie1, rating1) = ratings[0]
    (movie2, rating2) = ratings[1]
    return ((movie1, movie2), (rating1, rating2))

def filterDuplicates( userRatings ):
    ratings = userRatings[1]
    (movie1, rating1) = ratings[0]
    (movie2, rating2) = ratings[1]
    return movie1 < movie2

def computeCosineSimilarity(ratingPairs):
    numPairs = 0
    sum_xx = sum_yy = sum_xy = 0
    for ratingX, ratingY in ratingPairs:
        sum_xx += ratingX * ratingX
        sum_yy += ratingY * ratingY
        sum_xy += ratingX * ratingY
        numPairs += 1

    numerator = sum_xy
    denominator = sqrt(sum_xx) * sqrt(sum_yy)

    score = 0
    if (denominator):
        score = (numerator / (float(denominator)))

    return (score, numPairs)


conf = SparkConf()
sc = SparkContext(conf = conf)

print("\nLoading movie names...")
nameDict = loadMovieNames()

data = sc.textFile("s3n://sundog-spark/ml-1m/ratings.dat")

# Map ratings to key / value pairs: user ID => movie ID, rating
ratings = data.map(lambda l: l.split("::")).map(lambda l: (int(l[0]), (int(l[1]), float(l[2]))))

# Emit every movie rated together by the same user.
# Self-join to find every combination.
ratingsPartitioned = ratings.partitionBy(100)
joinedRatings = ratingsPartitioned.join(ratingsPartitioned)

# At this point our RDD consists of userID => ((movieID, rating), (movieID, rating))

# Filter out duplicate pairs
uniqueJoinedRatings = joinedRatings.filter(filterDuplicates)

# Now key by (movie1, movie2) pairs.
moviePairs = uniqueJoinedRatings.map(makePairs).partitionBy(100)

# We now have (movie1, movie2) => (rating1, rating2)
# Now collect all ratings for each movie pair and compute similarity
moviePairRatings = moviePairs.groupByKey()

# We now have (movie1, movie2) = > (rating1, rating2), (rating1, rating2) ...
# Can now compute similarities.
moviePairSimilarities = moviePairRatings.mapValues(computeCosineSimilarity).persist()

# Save the results if desired
moviePairSimilarities.sortByKey()
moviePairSimilarities.saveAsTextFile("movie-sims")

# Extract similarities for the movie we care about that are "good".
if (len(sys.argv) > 1):

    scoreThreshold = 0.97
    coOccurenceThreshold = 50

    movieID = int(sys.argv[1])

    # Filter for movies with this sim that are "good" as defined by
    # our quality thresholds above
    filteredResults = moviePairSimilarities.filter(lambda pairSim: \
        (pairSim[0][0] == movieID or pairSim[0][1] == movieID) \
        and pairSim[1][0] > scoreThreshold and pairSim[1][1] > coOccurenceThreshold)

    # Sort by quality score.
    results = filteredResults.map(lambda pairSim: (pairSim[1], pairSim[0])).sortByKey(ascending = False).take(10)

    print("Top 10 similar movies for " + nameDict[movieID])
    for result in results:
        (sim, pair) = result
        # Display the similarity result that isn't the movie we're looking at
        similarMovieID = pair[0]
        if (similarMovieID == movieID):
            similarMovieID = pair[1]
        print(nameDict[similarMovieID] + "\tscore: " + str(sim[0]) + "\tstrength: " + str(sim[1]))

## Section 6

### movie-recommendations-als-dataframe.py

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, LongType
from pyspark.ml.recommendation import ALS
import sys
import codecs

def loadMovieNames():
    movieNames = {}
    # CHANGE THIS TO THE PATH TO YOUR u.ITEM FILE:
    with codecs.open("E:/SparkCourse/ml-100k/u.ITEM", "r", encoding='ISO-8859-1', errors='ignore') as f:
        for line in f:
            fields = line.split('|')
            movieNames[int(fields[0])] = fields[1]
    return movieNames


spark = SparkSession.builder.appName("ALSExample").getOrCreate()

moviesSchema = StructType([ \
                     StructField("userID", IntegerType(), True), \
                     StructField("movieID", IntegerType(), True), \
                     StructField("rating", IntegerType(), True), \
                     StructField("timestamp", LongType(), True)])

names = loadMovieNames()

ratings = spark.read.option("sep", "\t").schema(moviesSchema) \
    .csv("file:///SparkCourse/ml-100k/u.data")

print("Training recommendation model...")

als = ALS().setMaxIter(5).setRegParam(0.01).setUserCol("userID").setItemCol("movieID") \
    .setRatingCol("rating")

model = als.fit(ratings)

# Manually construct a dataframe of the user ID's we want recs for
userID = int(sys.argv[1])
userSchema = StructType([StructField("userID", IntegerType(), True)])
users = spark.createDataFrame([[userID,]], userSchema)

recommendations = model.recommendForUserSubset(users, 10).collect()

print("Top 10 recommendations for user ID " + str(userID))

for userRecs in recommendations:
    myRecs = userRecs[1]  #userRecs is (userID, [Row(movieId, rating), Row(movieID, rating)...])
    for rec in myRecs: #my Recs is just the column of recs for the user
        movie = rec[0] #For each rec in the list, extract the movie ID and rating
        rating = rec[1]
        movieName = names[movie]
        print(movieName + str(rating))

### real-estate.py

In [ ]:
from __future__ import print_function

from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler

if __name__ == "__main__":

    # Create a SparkSession (Note, the config section is only for Windows!)
    spark = SparkSession.builder.appName("DecisionTree").getOrCreate()


    # Load up data as dataframe
    data = spark.read.option("header", "true").option("inferSchema", "true")\
        .csv("file:///SparkCourse/realestate.csv")

    assembler = VectorAssembler().setInputCols(["HouseAge", "DistanceToMRT", \
                               "NumberConvenienceStores"]).setOutputCol("features")

    df = assembler.transform(data).select("PriceOfUnitArea", "features")

    # Let's split our data into training data and testing data
    trainTest = df.randomSplit([0.5, 0.5])
    trainingDF = trainTest[0]
    testDF = trainTest[1]

    # Now create our decision tree
    dtr = DecisionTreeRegressor().setFeaturesCol("features").setLabelCol("PriceOfUnitArea")

    # Train the model using our training data
    model = dtr.fit(trainingDF)

    # Now see if we can predict values in our test data.
    # Generate predictions using our decision tree model for all features in our
    # test dataframe:
    fullPredictions = model.transform(testDF).cache()

    # Extract the predictions and the "known" correct labels.
    predictions = fullPredictions.select("prediction").rdd.map(lambda x: x[0])
    labels = fullPredictions.select("PriceOfUnitArea").rdd.map(lambda x: x[0])

    # Zip them together
    predictionAndLabel = predictions.zip(labels).collect()

    # Print out the predicted and actual values for each point
    for prediction in predictionAndLabel:
      print(prediction)


    # Stop the session
    spark.stop()

### spark-linear-regression.py

In [ ]:
from __future__ import print_function

from pyspark.ml.regression import LinearRegression

from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors

if __name__ == "__main__":

    # Create a SparkSession (Note, the config section is only for Windows!)
    spark = SparkSession.builder.appName("LinearRegression").getOrCreate()

    # Load up our data and convert it to the format MLLib expects.
    inputLines = spark.sparkContext.textFile("regression.txt")
    data = inputLines.map(lambda x: x.split(",")).map(lambda x: (float(x[0]), Vectors.dense(float(x[1]))))

    # Convert this RDD to a DataFrame
    colNames = ["label", "features"]
    df = data.toDF(colNames)

    # Note, there are lots of cases where you can avoid going from an RDD to a DataFrame.
    # Perhaps you're importing data from a real database. Or you are using structured streaming
    # to get your data.

    # Let's split our data into training data and testing data
    trainTest = df.randomSplit([0.5, 0.5])
    trainingDF = trainTest[0]
    testDF = trainTest[1]

    # Now create our linear regression model
    lir = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

    # Train the model using our training data
    model = lir.fit(trainingDF)

    # Now see if we can predict values in our test data.
    # Generate predictions using our linear regression model for all features in our
    # test dataframe:
    fullPredictions = model.transform(testDF).cache()

    # Extract the predictions and the "known" correct labels.
    predictions = fullPredictions.select("prediction").rdd.map(lambda x: x[0])
    labels = fullPredictions.select("label").rdd.map(lambda x: x[0])

    # Zip them together
    predictionAndLabel = predictions.zip(labels).collect()

    # Print out the predicted and actual values for each point
    for prediction in predictionAndLabel:
      print(prediction)


    # Stop the session
    spark.stop()

## Section 7

### structured-streaming.py

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Dec 18 09:15:05 2019

@author: Frank
"""

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession

from pyspark.sql.functions import regexp_extract

# Create a SparkSession (the config bit is only for Windows!)
spark = SparkSession.builder.config("spark.sql.warehouse.dir", "file:///C:/temp").appName("StructuredStreaming").getOrCreate()

# Monitor the logs directory for new log data, and read in the raw lines as accessLines
accessLines = spark.readStream.text("logs")

# Parse out the common log format to a DataFrame
contentSizeExp = r'\s(\d+)$'
statusExp = r'\s(\d{3})\s'
generalExp = r'\"(\S+)\s(\S+)\s*(\S*)\"'
timeExp = r'\[(\d{2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]'
hostExp = r'(^\S+\.[\S+\.]+\S+)\s'

logsDF = accessLines.select(regexp_extract('value', hostExp, 1).alias('host'),
                         regexp_extract('value', timeExp, 1).alias('timestamp'),
                         regexp_extract('value', generalExp, 1).alias('method'),
                         regexp_extract('value', generalExp, 2).alias('endpoint'),
                         regexp_extract('value', generalExp, 3).alias('protocol'),
                         regexp_extract('value', statusExp, 1).cast('integer').alias('status'),
                         regexp_extract('value', contentSizeExp, 1).cast('integer').alias('content_size'))

# Keep a running count of every access by status code
statusCountsDF = logsDF.groupBy(logsDF.status).count()

# Kick off our streaming query, dumping results to the console
query = ( statusCountsDF.writeStream.outputMode("complete").format("console").queryName("counts").start() )

# Run forever until terminated
query.awaitTermination()

# Cleanly shut down the session
spark.stop()

### top-urls.py

In [ ]:
from pyspark.sql import SparkSession

import pyspark.sql.functions as func

# Create a SparkSession (the config bit is only for Windows!)
spark = SparkSession.builder.appName("StructuredStreaming").getOrCreate()

# Monitor the logs directory for new log data, and read in the raw lines as accessLines
accessLines = spark.readStream.text("logs")

# Parse out the common log format to a DataFrame
contentSizeExp = r'\s(\d+)$'
statusExp = r'\s(\d{3})\s'
generalExp = r'\"(\S+)\s(\S+)\s*(\S*)\"'
timeExp = r'\[(\d{2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]'
hostExp = r'(^\S+\.[\S+\.]+\S+)\s'

logsDF = accessLines.select(func.regexp_extract('value', hostExp, 1).alias('host'),
                         func.regexp_extract('value', timeExp, 1).alias('timestamp'),
                         func.regexp_extract('value', generalExp, 1).alias('method'),
                         func.regexp_extract('value', generalExp, 2).alias('endpoint'),
                         func.regexp_extract('value', generalExp, 3).alias('protocol'),
                         func.regexp_extract('value', statusExp, 1).cast('integer').alias('status'),
                         func.regexp_extract('value', contentSizeExp, 1).cast('integer').alias('content_size'))

logsDF2 = logsDF.withColumn("eventTime", func.current_timestamp())

# Keep a running count of endpoints
endpointCounts = logsDF2.groupBy(func.window(func.col("eventTime"), \
      "30 seconds", "10 seconds"), func.col("endpoint")).count()

sortedEndpointCounts = endpointCounts.orderBy(func.col("count").desc())

# Display the stream to the console
query = sortedEndpointCounts.writeStream.outputMode("complete").format("console") \
      .queryName("counts").start()

# Wait until we terminate the scripts
query.awaitTermination()

# Stop the session
spark.stop()